# 1. Prostory jmen a obor platnosti proměnných
V tomto notebooku navážeme na práci s funkcemi a podíváme se, jak Python vyhledává jména proměnných.
Stejný název může v různých částech kódu odkazovat na jiný objekt podle aktuálního prostoru jmen.

V následujícím notebooku na to navážeme třídami, které mají vlastní jmenné prostory.


## 1.1 Typy prostorů jmen a pravidlo LEGB
Prostor jmen (`namespace`) je mapování názvů na objekty.

Python při hledání jména postupuje v pořadí LEGB:
- lokální (`local`) - aktuální funkce
- obalující (`enclosing`) - nadřazené zanořené funkce
- globální (`global`) - modul
- vestavěný (`builtins`) - vestavěná jména jazyka

Když jméno nenajde ani v jednom kroku, vyvolá `NameError`.
Přes `globals()` a `locals()` můžeme tyto prostory zobrazit jako slovníková mapování.


In [ ]:
nejaka_promenna = 1
print(type(globals()))
print("nejaka_promenna" in globals())
print(globals()["nejaka_promenna"])


In [ ]:
nejaka_promenna = 1

def ukaz_lokalni_prostor():
    nova_promenna_uvnitr_funkce = 1
    nejaka_promenna = 2
    print(type(locals()))
    print(locals())

ukaz_lokalni_prostor()


In [ ]:
import builtins

print(type(builtins))
print("len" in dir(builtins))


In [ ]:
# pokud jméno není v žádném prostoru, záměrně vyvoláme NameError
print(nedefinovana_promenna)


In [ ]:
# pokud vyrobíme novou proměnnou zde, bude v globálním prostoru
moje_promenna = 42
print(id(moje_promenna))
print(id(globals()['moje_promenna']))


In [ ]:
# ukázka globálního prostoru
moje_promenna = 42


def vytiskni_globalni():
    print(moje_promenna)


vytiskni_globalni()


In [ ]:
# ukázka lokálního prostoru
moje_promenna = 42


def vytiskni_lokalni():
    moje_promenna = 43
    print(moje_promenna)


vytiskni_lokalni()


In [ ]:
# ukázka obalujícího prostoru
moje_promenna = 42


def vnejsi_funkce():
    moje_promenna = 43

    def vnitrni_funkce():
        print(moje_promenna)

    vnitrni_funkce()


vnejsi_funkce()


## 1.2 Lokální prostor a `UnboundLocalError`
Lokální prostor vzniká při volání funkce, ne při její definici.

Pokud funkce do proměnné přiřazuje, Python ji považuje za lokální v celé funkci. Čtení před přiřazením pak skončí `UnboundLocalError`.


In [ ]:
# tato ukázka záměrně vyvolá UnboundLocalError
moje_promenna = 42


def ukaz_unboundlocalerror():
    print(moje_promenna)
    moje_promenna = 43
    print(moje_promenna)


ukaz_unboundlocalerror()


## 1.3 Práce s globálním prostorem
Z funkce můžeme měnit obsah mutovatelného globálního objektu (např. listu), aniž bychom použili `global`.

Jakmile ale chceme jméno přesměrovat na nový objekt, Python ho bez `global` bere jako lokální.


In [ ]:
muj_list = [1, 2, 3]


def uprav_globalni_list():
    print(muj_list)
    muj_list[0] = 42
    muj_list.append(4)


uprav_globalni_list()
print(muj_list)


Pro přepsání globální vazby je potřeba `global`.

V praxi je lepší se tomu spíš vyhýbat, protože to zhoršuje čitelnost a testovatelnost kódu.


In [ ]:
muj_list = [1, 2, 3]

print(id(muj_list))


def prepis_globalni_vazbu():
    global muj_list
    muj_list = [42, 2, 3]


prepis_globalni_vazbu()

print(id(muj_list))
print(muj_list)


## 1.4 `nonlocal` v zanořených funkcích
Pro obalující funkci platí stejný princip.
Pokud ve vnořené funkci potřebujeme přesměrovat jméno z obalujícího prostoru na jiný objekt, použijeme `nonlocal`.


In [ ]:
# ukázka nonlocal při přesměrování vazby
def vnejsi_funkce():
    zprava = "Ahoj"

    def vnitrni_funkce():
        nonlocal zprava
        zprava = "Ahoj světe"

    vnitrni_funkce()
    print(zprava)


vnejsi_funkce()


## 1.5 Obor platnosti v praxi
Následující ukázka shrnuje, že stejný název může existovat současně v různých scopech bez vzájemného přepisování.


In [ ]:
promenna = 42


def prvni_funkce():
    promenna = 43
    print("1:", promenna)

    def druha_funkce():
        promenna = 44
        print("2:", promenna)

    druha_funkce()
    print("3:", promenna)


print("4:", promenna)
prvni_funkce()
print("5:", promenna)


## 1.6 Výchozí hodnoty parametrů
Výchozí hodnoty parametrů se vyhodnotí jednou při definici funkce.

U mutovatelných typů (např. `list`) se proto změny mezi voláními sdílejí. Přiřazení na lokální proměnnou uvnitř funkce ale původní default nepřepíše.


In [ ]:
def pridej_hodnotu(hodnoty=[]):
    hodnoty.append(42)
    print(hodnoty)
    hodnoty = []


pridej_hodnotu()
pridej_hodnotu()
pridej_hodnotu([])
pridej_hodnotu()
pridej_hodnotu()


Na aktuální výchozí hodnoty parametrů se lze podívat přes atribut `__defaults__`.


In [ ]:
def ukaz_defaulty(hodnoty=[], text=""):
    hodnoty.append(42)
    text += "42"
    print(hodnoty)
    print(text)


print(ukaz_defaulty.__defaults__)
ukaz_defaulty()
print(ukaz_defaulty.__defaults__)
ukaz_defaulty()
print(ukaz_defaulty.__defaults__)
ukaz_defaulty()
print(ukaz_defaulty.__defaults__)
ukaz_defaulty()
print(ukaz_defaulty.__defaults__)
